# MNIST机器学习入门

## MNIST

是一个入门级的计算机视觉数据集，它包含各种手写数字图片

In [ ]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

## 手写数字图片

![](https://ws4.sinaimg.cn/large/006tKfTcgy1ftiraovwfcj318u0eu412.jpg)

我们把这个数组展开成一个向量，长度是 28x28 = 784

In [10]:
mnist.train.images.shape

(55000, 784)

![](https://ws3.sinaimg.cn/large/006tKfTcgy1ftirejcowfj31i00aswfz.jpg)

## 对应的标签

相对应的MNIST数据集的标签是介于0到9的数字，用来描述给定图片里表示的数字。为了用于这个教程，我们使标签数据是"one-hot vectors"。 一个one-hot向量除了某一位的数字是1以外其余各维度数字都是0。所以在此教程中，数字n将表示成一个只有在第n维度（从0开始）数字为1的10维向量。比如，标签0将表示成([1,0,0,0,0,0,0,0,0,0,0])。

In [11]:
mnist.train.labels.shape

(55000, 10)

![](https://ws2.sinaimg.cn/large/006tKfTcgy1ftireu53m0j31b608ywf1.jpg)

## Softmax回归介绍

- 我们知道MNIST的每一张图片都表示一个数字，从0到9。我们希望得到给定图片代表每个数字的概率。比如说，我们的模型可能推测一张包含9的图片代表数字9的概率是80%但是判断它是8的概率是5%（因为8和9都有上半部分的小圆），然后给予它代表其他数字的概率更小的值。

- 这是一个使用softmax回归（softmax regression）模型的经典案例。softmax模型可以用来给不同的对象分配概率。即使在之后，我们训练更加精细的模型时，最后一步也需要用softmax来分配概率。

<img src="https://ws1.sinaimg.cn/large/006tKfTcgy1ftirn39ckpj31e20y0agb.jpg" width="600px">

## Tensorflow

- 为了用python实现高效的数值计算，我们通常会使用函数库，比如NumPy，会把类似矩阵乘法这样的复杂运算使用其他外部语言实现。不幸的是，从外部计算切换回Python的每一个操作，仍然是一个很大的开销。如果你用GPU来进行外部计算，这样的开销会更大。用分布式的计算方式，也会花费更多的资源用来传输数据。

- TensorFlow也把复杂的计算放在python之外完成，但是为了避免前面说的那些开销，它做了进一步完善。Tensorflow不单独地运行单一的复杂计算，而是让我们可以先用图描述一系列可交互的计算操作，然后全部一起在Python之外运行。（这样类似的运行方式，可以在不少的机器学习库中看到。）

In [12]:
import tensorflow as tf

我们通过操作符号变量来描述这些可交互的操作单元，可以用下面的方式创建一个：

In [13]:
x = tf.placeholder(tf.float32, [None, 784])

x不是一个特定的值，而是一个占位符`placeholder`，我们在TensorFlow运行计算时输入这个值。我们希望能够输入任意数量的MNIST图像，每一张图展平成784维的向量。我们用2维的浮点数张量来表示这些图，这个张量的形状是`[None，784 ]`。（这里的`None`表示此张量的第一个维度可以是任何长度的。）

我们的模型也需要权重值和偏置量，当然我们可以把它们当做是另外的输入（使用占位符），但TensorFlow有一个更好的方法来表示它们：`Variable` 。 一个`Variable`代表一个可修改的张量，存在在TensorFlow的用于描述交互性操作的图中。它们可以用于计算输入值，也可以在计算中被修改。对于各种机器学习应用，一般都会有模型参数，可以用`Variable`表示。